need to:
- make it work for argonne files. make sure it detects that there is a barcode file and then incorperate it into split_libraries

# Make OTU Table from many file types

The purpose of this program is, using inputs, to take a variety of file types as an input and to output an OTU table and other analysis based on user preferences.

In [2]:
%reload_ext autoreload 
%autoreload 2

import zipfile
import os
import subprocess
import sys

from IPython.display import FileLinks, FileLink
from functools import partial
from os import chdir
from shutil import copy2

--det_seq_file_type input orig_seqes is file path to the original inputted sequences. calls apropriate function to convert sequences into one FASTA file. outputs path to final FASTA file.

In [3]:
def det_seq_file_type(orig_seqs, out_path):
    '''This function determines the input type of the sequence file, and converts that file into a single joined read fasta file.'''
    
    
    parts = orig_seqs.split("/")
    file_name= parts[-1]
    if ".fna" in file_name or ".fasta" in file_name or ".fa" in file_name or ".fsa" in file_name:
        return orig_seqs
    if ".zip" in file_name:
        return convert_zip(orig_seqs, out_path)
    if "." not in file_name:
        return convert_folder(orig_seqs, out_path)
    if ".sff" in file_name:
        return convert_sff(orig_seqs, out_path)
    if ".bam" in file_name:
        return convert_bam(orig_seqs, out_path)

--convert_zip input orig_seqes is file path to the original inputted sequences, where they are in a zip folder. unzips file and calls det_seq_file_type() on unzipped file. outputs path to final FASTA file

In [4]:
def convert_zip(orig_seqs, out_path):
    parts = orig_seqs.split("/")
    file_name = parts[-1]
    file_location = "/".join(parts[:-1])
    
    unzipped_folder_name = file_name[:-4]
    
    
    zfile = zipfile.ZipFile(orig_seqs)
    zfile.extractall(out_path + '/' + unzipped_folder_name)
            
    joined_data = det_seq_file_type(out_path + '/' + unzipped_folder_name, out_path)
    
    return joined_data

3 cases:
1. R1 and R2 in a folder
2. R1, R2 and barcode
3. subfolders, each containing R1 and R2 (multiple_joined_paired_ends)

--convert_folder input orig_seqes is file path to the original inputted sequences, where this is a folder containing R1 and R2 reads, and possibly a file containing barcodes. outputs path to final FASTA file.

In [5]:
def convert_folder(orig_seqs, out_path):
    files_in_folder = os.listdir(orig_seqs)

    r1_files = 0
    for f in files_in_folder:
        if os.path.isdir(orig_seqs +'/'+ f):
            return multiple_joins(orig_seqs, out_path)
        elif not os.path.isdir(orig_seqs+ '/' + f):
            if 'R1' in f:
                r1_files+=1
    if r1_files > 1:
        return multiple_joins(orig_seqs, out_path)

    r_one = ""
    r_two = ""
    barcodes = ""
    joined_data = ""

    parts = orig_seqs.split("/")
    file_location = "/".join(parts[:-1])
    file_name = parts[-1]

    for f in files_in_folder:
        if "R1" in f or "r1" in f:
            r_one = f
        elif "R2" in f or "r2" in f:
            r_two = f
        elif "barcodes" or "Barcodes" or 'I1' in f:
            barcodes = f
            

    if barcodes == "":
        forward = orig_seqs+ '/' + r_one
        reverse = orig_seqs+ '/' + r_two

        !join_paired_ends.py -f $forward -r $reverse -o $out_path"/joined_data"
    else:
        barc = orig_seqs + '/' + barcodes
        forward = orig_seqs+ '/' + r_one
        reverse = orig_seqs+ '/' + r_two
        
        !join_paired_ends.py -f $forward -r $reverse -b $barc -o $out_path'/joined_data'


    return out_path + '/' + "joined_data"

case 3

--multiple_joins input orig_seqes is file path to the original inputted sequences, where this is a folder containing subfolders, where each subfolder contains two files: R1 and R2 reads of the same sequence. outputs path to final FASTA file

In [6]:
def multiple_joins(orig_seqs, out_path):
    #Join the forward and reverse sequences
    #Works for subfolders and mutiple files within one folder
    !multiple_join_paired_ends.py -i $orig_seqs -o $out_path"/joined_data"

    return out_path + '/' + "joined_data"

converts sff file, outputs path to FASTA file

In [7]:
def convert_sff(orig_seqs, out_path):
    parts = orig_seqs.split("/")
    file_location = "/".join(parts[:-1])
    
    !process_sff -i $orig_seqs -o $out_path"/joined_data"
    return out_path+ '/' + "joined_data"

NEED TO FIGURE OUT --convert_bam input orig_seqes is file path to the original inputted sequences, where this is an bam file. outputs path to final FASTA file.

In [8]:
# def convert_bam(orig_seqs, out_path):
#     bedtools bamtofastq [OPTIONS] -i orig_seqs -fq Fasting_Example.fna
#     http://bedtools.readthedocs.io/en/latest/content/tools/bamtofastq.html

In [18]:
def run_pipeline(seqs_final, out_path, username, clustering_method, otufasta97, otutext97,gg_13_5_fasta, gg_13_5_txt, mapping_path):
    
#     Remove the non-joined sequences. different for whether or not there are multiple libraries 

    in_joined_data = os.listdir(seqs_final)

    n_dirs = 0
    for f in in_joined_data:
        if os.path.isdir(seqs_final+ '/' + f):
            n_dirs+=1
            to_delete_un1 = seqs_final + '/' + f +"/fastqjoin.un1.fastq"
            to_delete_un2 = seqs_final + '/'  + f +"/fastqjoin.un2.fastq"
            !rm -r $to_delete_un1
            !rm -r $to_delete_un2

    if n_dirs == 0:
        !rm -r $seqs_final"/fastqjoin.un1.fastq"
        !rm -r $seqs_final"/fastqjoin.un2.fastq"

    # We are going to concatenate all the sequences into 1 to run 
    # Note the folder for the next step cannot contain this file.
    
    in_joined_data_2 = os.listdir(seqs_final)
    filenames = []
    
    for f in in_joined_data_2:
        if "_L001_R1_001" in f:
            filenames.append(seqs_final + '/' + f + '/fastqjoin.join.fastq')
            

    #Rename files in Python
    chdir(seqs_final)
    file_names = os.listdir(seqs_final)
    for ifile in file_names:
        os.rename(ifile, str.replace(ifile, "_", ""))
        


    #Split libraries. includes saving the fasta files (see qiime parameters) using qiime. Determines how many files (more than 1 R1 or R2), split vs. multiple split.
##should have an option to use a barcodes file (-b option in split_libraries.py)
    
    in_joined_data = os.listdir(seqs_final) 
    fastq_joined = in_joined_data[0]
    n_dirs = 0
    for f in in_joined_data:
        if os.path.isdir(seqs_final+ '/' + f):
            n_dirs+=1
    if n_dirs > 1:
##Need to have option to use mapping files
        print('hi')
        !multiple_split_libraries_fastq.py -i '/Users/user/barengolts_otus_1/joined_data' -o $out_path'/split_data' --demultiplexing_method sampleid_by_file --include_input_dir_path --remove_filepath_in_name -p $out_path'/qiime_parameters.txt'
    else:
        input_seqs = seqs_final + '/' + fastq_joined
        !split_libraries_fastq.py -i $input_seqs -o $out_path'/split_data' -b $out_path'/joined_data/fastqjoin.joinbarcodes.fastq' -m $mapping_path
    
    
    #Fixing the names
    chdir(out_path+'/split_data')
    #!sed 's/L001R1001//g' seqs.fastq > seqs_fixed.fastq 
    !sed 's/L001R1001//g' seqs.fna > seqs_fixed.fna

##reference database is gg_13_8
    !identify_chimeric_seqs.py -i $out_path'/split_data/seqs_fixed.fna' -m usearch61 -o $out_path'/usearch_checked_chimeras/' -r $otufasta97

    !filter_fasta.py -f $out_path'/split_data/seqs_fixed.fna' -o $out_path'/seqs_nochimeras_filtered.fna' -s $out_path'/usearch_checked_chimeras/chimeras.txt' -n
    
    
    if clustering_method == "denovo" or clustering_method == "Denovo":
        !pick_otus.py -i $out_path'/seqs_nochimeras_filtered.fna' -o $out_path'/OTUclustering_denovo' -m uclust
        otu_clust_path = '/OTUclustering_denovo'
        !make_otu_table.py -i $out_path'/OTUclustering_denovo/seqs_nochimeras_filtered_otus.txt' -o $out_path$otu_clust_path'/otu_table_denovo.biom'
        !biom convert -i $out_path$otu_clust_path'/otu_table_denovo.biom' -o $out_path'/otu_table_'$clustering_method'.txt' --table-type="OTU table" --to-tsv 
        !biom convert -i $out_path$otu_clust_path'/otu_table_denovo.biom' -o $out_path'/otu_table_'$clustering_method'.json' --table-type="OTU table" --to-json
        !mkdir $out_path'/biom_summarize' 
        !biom summarize-table -i $out_path$otu_clust_path'/otu_table_denovo.biom' -o $out_path'/biom_summarize/otu_table_'$clustering_method'_sum.txt'
        !biom summarize-table -i $out_path$otu_clust_path'/otu_table_denovo.biom' -o $out_path'/biom_summarize/otu_table_'$clustering_method'_sumq.txt'  --qualitative
        
    elif clustering_method == "open" or clustering_method == "Open":
        !pick_open_reference_otus.py -i $out_path'/seqs_nochimeras_filtered.fna' -o $out_path'/OTUclustering_openreference_usearch' --reference_fp $otufasta97 -f -m usearch61 -p $out_path'/qiime_parameters.txt'
        otu_clust_path = '/OTUclustering_openreference_usearch'
        
    elif clustering_method == "closed" or clustering_method == "Closed":  
        !pick_closed_reference_otus.py -i $out_path'/seqs_nochimeras_filtered.fna' -o $out_path'/OTUclustering_closedreference' --reference_fp $otufasta97 -f -m usearch61 -p $out_path'/qiime_parameters.txt'
        otu_clust_path = '/OTUclustering_closedreference'
    
    if clustering_method != "denovo" or clustering_method != "Denovo":
        !biom convert -i $out_path$otu_clust_path'/otu_table_mc2_w_tax.biom' -o $out_path'/otu_table_'$clustering_method'.txt' --table-type="OTU table" --to-tsv 

        !biom convert -i $out_path$otu_clust_path'/otu_table_mc2_w_tax.biom' -o $out_path'/otu_table_'$clustering_method'.json' --table-type="OTU table" --to-json

        # Summarize tables
        !mkdir $out_path'/biom_summarize' 
        !biom summarize-table -i $out_path$otu_clust_path'/otu_table_mc2_w_tax.biom' -o $out_path'/biom_summarize/otu_table_'$clustering_method'_sum.txt'
        !biom summarize-table -i $out_path$otu_clust_path'/otu_table_mc2_w_tax.biom' -o $out_path'/biom_summarize/otu_table_'$clustering_method'_sumq.txt'  --qualitative
    
    #======================================================================================================================
    #PICRUST
    #======================================================================================================================
    


In [17]:
#What is the name of your user profile on your computer? The outputs from this program will be located in your user folder.
username = 'user'

##What is the name of the output folder you would like to create?
output_folder = "barengolts_otus_2"

##What is the path to macqiime on your computer, including the 'macqiime' folder.
qiime_path = '/macqiime'

out_path =  "/Users"+ "/" + username + "/" + output_folder
if not os.path.exists(out_path):
    os.makedirs(out_path)


#orig_seqs is the path to the original inputted sequence, including the file name
## What is the path to the DNA sequencing file, including the file name? 
orig_seqs = '/Users/user/Desktop/Miseq158_Barengolts_13228_ReRUN-32930911_2'

#What is the path to your mapping file?
mapping_path = ''

#seqs_final is path to final sequencing file, including the file name
seqs_final = det_seq_file_type(orig_seqs, out_path)

##What method would you like to use for clustering (denovo, open, closed)?
clustering_method = 'open'

## What platform type are you using? Illumina or 454?    
platform_type = 'Illumina'

##What is the path to gg_13_5?
gg_13_5_path = '/Users/user/Desktop/Independent_Study/gg_13_5_otus'

gg_13_5_fasta = gg_13_5_path + '/rep_set/97_otus.fasta'
gg_13_5_txt = gg_13_5_path + '/taxonomy/97_otu_taxonomy.txt'

#dont worry about this stuff rn
otufasta97 = '/Users/user/Desktop/Independent_Study/gg_13_8_otus/rep_set/97_otus.fasta'

otutxt97  = '/Users/user/Desktop/Independent_Study/gg_13_8_otus/taxonomy/97_otu_taxonomy.txt'



In [19]:
##Use the below section to if you would like to set custom parameters
##If you do not want to set custom parameters, the default QIIME parameters will be used

#Split_libraries_fastq parameters
max_bad_run_length = 3
quality_threshold = 30
sequence_max_n = 1
store_demultiplexed_fastq = True
#Pick_otus parameters 
similarity = .9
# valid usearch_sort_methods include "abundance", "length", or "None"
usearch61_sort_method = "abundance"
sizeorder = True


parameter_file = open(out_path + "/qiime_parameters.txt", "w+")

parameter_file.write("# Parameters for defining split_libraries and pick_otus \n")
parameter_file.write(("split_libraries_fastq:max_bad_run_length ") + str(max_bad_run_length) + " \n")
parameter_file.write(('split_libraries_fastq:phred_quality_threshold ') + str(quality_threshold) + " \n")
parameter_file.write(("split_libraries_fastq:sequence_max_n ") + str(sequence_max_n) + " \n")
parameter_file.write(("split_libraries_fastq:store_demultiplexed_fastq") + str(store_demultiplexed_fastq) + " \n")
parameter_file.write(("pick_otus:similarity ") + str(similarity) + " \n")
parameter_file.write(("pick_otus:usearch61_sort_method ") + usearch61_sort_method + " \n")
parameter_file.write(("pick_otus:sizeorder ") + str(sizeorder))

parameter_file.close()

In [20]:
#run through the pipeline
run_pipeline(seqs_final, out_path, username, clustering_method, otufasta97, otutxt97, gg_13_5_fasta, gg_13_5_txt, mapping_path)

hi
Traceback (most recent call last):
  File "/macqiime/anaconda/bin/multiple_split_libraries_fastq.py", line 219, in <module>
    main()
  File "/macqiime/anaconda/bin/multiple_split_libraries_fastq.py", line 216, in main
    close_logger_on_success=True)
  File "/macqiime/anaconda/lib/python2.7/site-packages/qiime/workflow/util.py", line 122, in call_commands_serially
    raise WorkflowError(msg)
qiime.workflow.util.WorkflowError: 

*** ERROR RAISED DURING STEP: split_libraries_fastq.py
Command run was:
 split_libraries_fastq.py --phred_quality_threshold 30 --max_bad_run_length 3 --sequence_max_n 1 -i /Users/user/barengolts_otus_1/joined_data/Barengolts37-2041S261L001R1001/fastqjoin.join.fastq,/Users/user/barengolts_otus_1/joined_data/Barengolts80-3034S304L001R1001/fastqjoin.join.fastq,/Users/user/barengolts_otus_1/joined_data/Barengolts19-2023S243L001R1001/fastqjoin.join.fastq,/Users/user/barengolts_otus_1/joined_data/Barengolts66-3020S290L001R1001/fastqjoin.join.fastq,/Users/user/b

In [ ]:
def use_picrust(seqs_final, out_path, username, clustering_method, otufasta97, otutext97,gg_13_5_fasta, gg_13_5_txt):
    
    #Use the version of GG available from PICRUST gg_13_5
    !identify_chimeric_seqs.py -i $out_path'/split_data/seqs_fixed.fna' -m usearch61 -o $out_path'/PICRUST_outputs/usearch_checked_chimeras/' -r $gg_13_5_fasta
    
    !filter_fasta.py -f $out_path'/split_data/seqs_fixed.fna' -o $out_path'/PICRUST_outputs/seqs_nochimeras_filtered.fna' -s $out_path'/PICRUST_outputs/usearch_checked_chimeras/chimeras.txt' -n
    
    #create OTU table without chimeras for PICRUST. Must use open reference.
    output_otu = '/PICRUST_outputs/OTUclustering_closed'
    !pick_otus.py -i $out_path'/PICRUST_outputs/seqs_nochimeras_filtered.fna' -o $out_path$output_otu --refseqs_fp $gg_13_5_fasta -m usearch61_ref  --suppress_new_clusters
    
    !make_otu_table.py -i $out_path$output_otu'/seqs_nochimeras_filtered_otus.txt' --taxonomy $gg_13_5_txt -o $out_path'/PICRUST_outputs/otu_table.biom'
    
    !pick_rep_set.py -i $out_path$output_otu'/seqs_nochimeras_filtered_otus.txt' -o $out_path'/PICRUST_outputs/rep_set.fna' -f $out_path"/PICRUST_outputs/seqs_nochimeras_filtered.fna"

    !assign_taxonomy.py -i $out_path'/PICRUST_outputs/rep_set.fna' -o $out_path'/AssignTaxa' -r $gg_13_5_fasta -t $gg_13_5_txt
    
    #Align sequences in Qiime (Takes long for large datasets) 
    !align_seqs.py -i $out_path'/PICRUST_outputs/rep_set.fna' -o $out_path'/PICRUST_outputs/RepSeqAligmenment' -t $'/Users/user/Desktop/Independent_Study/gg_13_5_otus/rep_set_aligned/97_otus.fasta'
    
    #Filter alignments 
    !filter_alignment.py -i $out_path'/PICRUST_outputs/RepSeqAligmenment/rep_set_aligned.fasta' -o $out_path'/PICRUST_outputs/FilterAlignment'
    
    #Make phylogeny -Same results with 85 than 97
    !make_phylogeny.py -i $out_path'/PICRUST_outputs/FilterAlignment/rep_set_aligned_pfiltered.fasta' -o $out_path'/PICRUST_outputs/refset.tree'
    
    !biom convert -i $out_path'/PICRUST_outputs/otu_table.biom' -o $out_path'/PICRUST_outputs/otu_table.txt' --table-type="OTU table" --to-tsv 
    
    !biom convert -i $out_path'/PICRUST_outputs/otu_table.biom' -o $out_path'/PICRUST_outputs/otu_table.json' --table-type="OTU table" --to-json
    
    # Summarize tables (quantitative and qualitative)
    !mkdir $out_path'/PICRUST_outputs/biom_summarize'
    !biom summarize-table -i $out_path'/PICRUST_outputs/otu_table.biom' -o $out_path'/PICRUST_outputs/biom_summarize/otu_table_sum.txt'
    !biom summarize-table -i $out_path'/PICRUST_outputs/otu_table.biom' -o $out_path'/PICRUST_outputs/biom_summarize/otu_table_sumq.txt'  --qualitative 

    #Normalize by copy number
    # This part is done locally
    !normalize_by_copy_number.py -i $out_path'/PICRUST_outputs/otu_table.biom'  -o $out_path'/PICRUST_outputs/norm.biom'
    
    #Predict metagenomes
    !predict_metagenomes.py -i $out_path'/PICRUST_outputs/norm.biom' -o $out_path'/PICRUST_outputs/metagenome_predictions.biom' --with_confidence
    
    #Categorize by function with level 2 in KEGG (.biom and .txt)
    !categorize_by_function.py -i $out_path'/PICRUST_outputs/metagenome_predictions.biom' -c KEGG_Pathways -l 2 -o $out_path'/PICRUST_outputs/predicted_metagenomes.L2.biom'
    !categorize_by_function.py -i $out_path'/PICRUST_outputs/metagenome_predictions.biom' -c KEGG_Pathways -l 2 -o $out_path'/PICRUST_outputs/predicted_metagenomes.L2.txt' -f 

    #Categorize by function with level 3 in KEGG (.biom and .txt)
    !categorize_by_function.py -i $out_path'/PICRUST_outputs/metagenome_predictions.biom' -c KEGG_Pathways -l 3 -o $out_path'/PICRUST_outputs/predicted_metagenomes.L3.biom'
    !categorize_by_function.py -i $out_path'/PICRUST_outputs/metagenome_predictions.biom' -c KEGG_Pathways -l 3 -o $out_path'/PICRUST_outputs/predicted_metagenomes.L3.txt' -f
    
    #Prediction of functions by OTUs
    !metagenome_contributions.py -i $out_path'/PICRUST_outputs/norm.biom'  -o $out_path'/PICRUST_outputs/ko_metagenome_contributions.tab'

In [ ]:
help(convert_zip)

- help include:
- what the function does  (2-3 lines)
- what are the inputs
- what are the outputs


make it cleaner

In [ ]:
from shutil import copy2

In [ ]:
copy2('/Users/user/Desktop/argonne_files/Undetermined_S0_L001_I1_001.fastq.gz','/Users/user/Desktop/hi')